<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot véhicule dans la phrase respective.  Les méthodes sont les suivantes (elles seront décrites plus précisément plus bas) :

1. Algorithme de Lesk simplifié.
1. Utilisation de word2vec pour la similarité contexte vs. synset.
1. Classification supervisée utilisant des traits lexicaux, avec deux représentations :
  1. les mots en position -1, -2, ..., et +1, +2, ..., par rapport à *interest* ;
  1. apparition de mots indicateurs dans le voisinage de *interest*.

Les deux premières méthodes peuvent être qualifiées de *non supervisées*, mais en réalité elles n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d’une occurrence de *interest* avec les définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L’algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  

La troisième méthode vise à classifier les occurrence de *interest* en utilisant les sens comme des classes, et en utilisant comme traits les mots du contexte.  Vous utiliserez des méthodes d'apprentissage supervisé, en divisant les données en sous-ensembles d'apprentissage et de test.

## 0. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html).  Il se trouve en bas de cette page.  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre aux questions suivantes :

1. Veuillez recopier l'URL du fichier ZIP et celle du fichier `README.int.txt`.
2. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?  Considère-t-on les pluriels aussi ?  Que se passe-t-il si une phrase contient plusieurs occurrences ?

In [ ]:
# 1. ZIP: https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
#    README.int.txt: https://www.d.umn.edu/~tpederse/Data/README.int.txt
# 2. Format: Chaque phrase est séparé par des `$$`
#    Annotation: Pour indiquer le sens, un numéro est ajouté après le mots `interest`. (e.g. interest_6)
#                Sens possible:
#                Sense 1 - readiness to give attention
#                Sense 2 - quality of causing attention to be given to
#                Sense 3 - activity, etc. that one gives attention to
#                Sense 4 - advantage, advancement or favor
#                Sense 5 - a share in a company or business
#                Sense 6 - money paid for the use of money
#    Pluriels: Le pluriel est pris en compte!
#    Plusieurs occurrences: Seulement une occurence à l'identifieur, les autres sont précédé par une `*` (e.g. *interest)

3. D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait.

In [ ]:
# Sense 1 =  361 occurrences (15%) - readiness to give attention
# Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
# Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
# Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
# Sense 5 =  500 occurrences (21%) - a share in a company or business
# Sense 6 = 1252 occurrences (53%) - money paid for the use of money

4. De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés (écrire p.ex., Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying").

In [ ]:
# Les sens viennent de Longman Dictionary of Contemporary English
# https://www.ldoceonline.com/
#
# Sense 1 = "if you have an interest in something or someone, you want to know or learn more about them"
# Sense 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
# Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
# Sense 4 = "the things that bring advantages to someone or something"
# Sense 5 = "if you have an interest in a particular company or industry, you own shares in it"
# Sense 6 = "the extra money that you must pay back when you borrow money"

5. En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Alignez les **définitions** de ces synsets avec les six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [ ]:
# Il y a 7 synsets sur WordNet
# Synsets 1 = "a sense of concern with and curiosity about someone or something"
# Synsets 2 = "a reason for wanting something done"
# Synsets 3 = "the power of attracting or holding one's attention (because it is unusual or exciting etc.)"
# Synsets 4 = "a fixed charge for borrowing money; usually a percentage of the amount borrowed"
# Synsets 5 = "(law) a right or legal share of something; a financial involvement with something"
# Synsets 6 = "(usually plural) a social group whose members control some field of activity and who have common aims"
# Synsets 7 = "a diversion that occupies one's time and thoughts (usually pleasantly)"


# Sense 1 = Synset 1
# Sense 2 = Synset 3
# Sense 3 = Synset 7
# Sense 4 = Synset 2
# Sense 5 = Synset 5
# Sense 6 = Synset 4

# Le synset 6 ne match avec aucun des sens que nous avons car celui-ci parle de `interest` dans son sens pluriel

6. Définissez (manuellement, ou avec un peu de code Python) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [1]:
import nltk

In [2]:
from random import randrange

In [3]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
senses1 = [
    ["readiness", "give", "attention"],
    ["quality" ,"causing", "attention", "given"],
    ["activity", "attention"],
    ["advantage", "advancement", "favor"],
    ["share", "company", "business"],
    ["money", "paid"]
]

print(senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention', 'given'], ['activity', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid']]


7. En combinant les définitions obtenues aux points (3) et (4) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** qui le caractérise.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [78]:
senses2 = [
    ["readiness", "give", "attention", "interest", "something", "someone", "want", "know", "learn", "about", "them"],
    ["quality" ,"causing", "attention", "given", "feature", "something", "attracts", "makes", "want", "know", "more", "about"],
    ["activity", "attention", "activity", "enjoy", "doing", "subject", "studying"],
    ["advantage", "advancement", "favor", "things", "bring", "advantages", "someone", "something"],
    ["share", "company", "business", "interest", "particular", "company", "industry", "shares"],
    ["money", "paid", "extra", "must", "back", "borrow"]
]

print(senses2)

[['readiness', 'give', 'attention', 'interest', 'something', 'someone', 'want', 'know', 'learn', 'about', 'them'], ['quality', 'causing', 'attention', 'given', 'feature', 'something', 'attracts', 'makes', 'want', 'know', 'more', 'about'], ['activity', 'attention', 'activity', 'enjoy', 'doing', 'subject', 'studying'], ['advantage', 'advancement', 'favor', 'things', 'bring', 'advantages', 'someone', 'something'], ['share', 'company', 'business', 'interest', 'particular', 'company', 'industry', 'shares'], ['money', 'paid', 'extra', 'must', 'back', 'borrow']]


8. Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Les phrases sont-elles déjà tokenisées ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en quatre comme indiqué.

In [119]:
# Veuillez répondre ici à la question.
sentences = []

with open("interest-original.txt", 'r', encoding='utf-8') as fd:
    lines = fd.readlines()
    
    for line in lines:
        clean = line.replace('$$', '').replace('=', '').replace('.', '').replace(',', '').replace('\'', '')
        if clean == "\n":
            continue
        sentences.append(nltk.tokenize.word_tokenize(clean))

        
print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

Il y a 2368 phrases.
En voici 3 au hasard :
[['investor', 'interest_1', 'in', 'stock', 'funds', '``', 'has', 'nt', 'stalled', 'at', 'all', 'mr', 'hines', 'maintains'], ['it', 'is', 'in', 'the', 'western', 'interest_4', 'to', 'see', 'mr', 'gorbachev', 'succeed'], ['revco', 'insists', 'that', 'the', 'proposal', 'is', 'simply', 'an', '``', 'expression', 'of', 'interest_1', 'because', 'under', 'chapter', '11', 'revco', 'has', '``', 'exclusivity', 'rights', 'until', 'feb', '28']]


## 1. Algorithme de Lesk simplifié

Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne le numéro du sens le plus probable selon l'algorithme de Lesk.  On rappelle que cet algorithme choisit le sens qui a le maximum de mots en commun avec le voisinage de *interest*.  Vous pouvez choisir vous-même la taille de ce voisinage (*window_size*).  En cas d'égalité entre deux sens, tirer la réponse au sort.

*Note : il ne serait pas correct de choisir le sens le plus fréquent en cas d'égalité, car cela suppose qu'on a utilisé les données de test pour calculer ces fréquences, donc pour s'entraîner, ce qui est incorrect !*

In [81]:
import random
import re

In [198]:
def wsd_lesk(senses, sentence, window_size=2):
    interest_pos = [i for i, w in enumerate(sentence) if re.match('^interest[s]?_[1-6]$', w)][0]
    neighbourhood_start = max(interest_pos-window_size, 0)
    neighbourhood_end = min(interest_pos+window_size, len(sentence)-1)
    
    neighbours = sentence[neighbourhood_start:interest_pos] + sentence[interest_pos+1:neighbourhood_end+1]

    score = [0] * len(senses)
    for i, sense in enumerate(senses):
        score[i] = len([n for n in neighbours if n in sense])

    best = max(score)
    likely_senses = []
    for i, s in enumerate(score):
        if s == best:
            likely_senses.append(i)

    return random.choice(likely_senses) + 1

Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la fonction de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des sens et la liste de phrases, et retourne le score total de la méthode (pourcentage du nombre de réponses correctes par phrase) en vérifiant pour chaque phrase si le sens trouvé est identique au sens annoté.  Améliorez ensuite la méthode pour afficher le taux de correction par sens.

In [196]:
def evaluate_wsd(fct_name, senses, sentences, window_size=2):
    nb_correct = 0
    nb_total = 0
    
    per_sense = [(0,0)] * len(senses)
    for sentence in sentences:
        system_sense = -1
        for i, w in enumerate(sentence):
            if re.match('^interest[s]?_[1-6]$', w):
                system_sense = int(w[-1])
                break

        nb_total += 1
        calculated = fct_name(senses, sentence, window_size)
        per_sense[system_sense-1] = (per_sense[system_sense-1][0], per_sense[system_sense-1][1]+1)
        
        if calculated == system_sense:
            nb_correct += 1
            per_sense[system_sense-1] = (per_sense[system_sense-1][0]+1, per_sense[system_sense-1][1])
    
    return (nb_correct, nb_total), per_sense

In [210]:
window_size = 18
results_senses1 = evaluate_wsd(wsd_lesk, senses1, sentences, window_size)
results_senses2 = evaluate_wsd(wsd_lesk, senses2, sentences, window_size)

print("Wsd_leks, window: ", window_size)
print("---")
print("Senses 1:")
print("Global: ", results_senses1[0][0]*100/results_senses1[0][1], "%")
for i, res in enumerate(results_senses1[1]):
    print("Sense ", i+1, ": ", res[0]/res[1]*100)

print()

print("Senses 2:")
print("Global: ", results_senses2[0][0]*100/results_senses2[0][1], "%")
for i, res in enumerate(results_senses2[1]):
    print("Sense ", i+1, ": ", res[0]/res[1]*100)

Wsd_leks, window:  18
---
Senses 1:
Global:  20.988175675675677 %
Sense  1 :  9.695290858725762
Sense  2 :  36.36363636363637
Sense  3 :  6.0606060606060606
Sense  4 :  18.53932584269663
Sense  5 :  34.599999999999994
Sense  6 :  19.808306709265175

Senses 2:
Global:  22.635135135135137 %
Sense  1 :  17.174515235457065
Sense  2 :  27.27272727272727
Sense  3 :  10.606060606060606
Sense  4 :  15.730337078651685
Sense  5 :  37.6
Sense  6 :  19.808306709265175


En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur taux de correction de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

*Note : optimiser la taille de la fenêtre sur les données de test serait incorrect !*

In [74]:
# Veuillez répondre ici à la question.
# Les listes donne environ les même scores, mais globalement `senses2` est meilleur
# Avec une fenêtre de 18 & senses2 on obtiens un résultat de ~22%

## 2. Utilisation de word2vec pour la similarité contexte vs. synset

En réutilisant une partie du code de `wsd_lesk`, définissez maintenant une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec**.  On vous encourage à chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.

Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.  La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*. Vous pouvez choisir la taille de ce voisinage (`window_size`).  En cas d'égalité, tirer le sens au sort.

In [213]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
wv_from_bin = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

/home/ducky/.conda/envs/tal/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [221]:
# Veuillez répondre ici à la question.
def wsd_word2vec(senses, sentence, window_size=2):
    interest_pos = [i for i, w in enumerate(sentence) if re.match('^interest[s]?_[1-6]$', w)][0]
    neighbourhood_start = max(interest_pos-window_size, 0)
    neighbourhood_end = min(interest_pos+window_size, len(sentence)-1)
    
    neighbours = sentence[neighbourhood_start:interest_pos] + sentence[interest_pos+1:neighbourhood_end+1]

    score = [0] * len(senses)
    for i, sense in enumerate(senses):
        try:
            score[i] = wv_from_bin.n_similarity(neighbours, sense) #len([n for n in neighbours if n in sense])
        except:
            continue

    best = max(score)
    likely_senses = []
    for i, s in enumerate(score):
        if s == best:
            likely_senses.append(i)

    return random.choice(likely_senses) + 1

Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez les scores (globaux et par sens) pour la similarité word2vec.  Comment se comparent-ils avec les précédents ?

Il est equivalent à la méthode `evaluate_wsd`, si ce n'est légerement meilleur.

In [224]:
window_size = 2
results_senses1 = evaluate_wsd(wsd_word2vec, senses1, sentences, window_size)
results_senses2 = evaluate_wsd(wsd_word2vec, senses2, sentences, window_size)

print("Word2vec, window: ", window_size)
print("---")
print("Senses 1:")
print("Global: ", results_senses1[0][0]*100/results_senses1[0][1], "%")
for i, res in enumerate(results_senses1[1]):
    print("Sense ", i+1, ": ", res[0]/res[1]*100)

print()

print("Senses 2:")
print("Global: ", results_senses2[0][0]*100/results_senses2[0][1], "%")
for i, res in enumerate(results_senses2[1]):
    print("Sense ", i+1, ": ", res[0]/res[1]*100)

Word2vec, window:  2
---
Senses 1:
Global:  20.945945945945947 %
Sense  1 :  12.742382271468145
Sense  2 :  18.181818181818183
Sense  3 :  4.545454545454546
Sense  4 :  12.921348314606742
Sense  5 :  30.4
Sense  6 :  21.56549520766773

Senses 2:
Global:  25.760135135135137 %
Sense  1 :  21.329639889196674
Sense  2 :  36.36363636363637
Sense  3 :  9.090909090909092
Sense  4 :  9.550561797752808
Sense  5 :  26.8
Sense  6 :  29.712460063897762


## 3. Classification supervisée avec des traits lexicaux
Dans cette partie du labo, vous entraînerez des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le principal défi sera de transformer chaque phrase en un ensemble de traits, pour créer les données en vue des expériences de classification.

Vous utiliserez le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  (Il existe de nombreux autres classifieurs supervisés, par exemple dans la boîte à outils `scikit-learn`.)

De plus, vous devrez séparer les 2368 occurrences en ensembles d'entraînement et de test.

### 3.A. Traits lexicaux positionnels

Dans cette première représentation des traits, vous les coderez comme `mot-2`, `mot-1`, `mot+1`, `mot+2`, etc. (fenêtre de taille `2*window_size` autour de *interest*) et vous leur donnerez les valeurs des mots observés aux emplacements respectifs, ou alors `NONE` si la fenêtre dépasse la limite de la phrase.  Vous ajouterez un trait qui est le mot *interest* lui-même, qui peut être au singulier ou au pluriel.  Pour chaque occurrence de *interest*, vous devez donc générer une représentation formelle avec un dictionnaire Python puis un entier :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```
où l'entier est le numéro du sens (ici, 6).  Cette valeur servira à l'entraînement, puis elle sera cachée à l'évaluation, et la prédiction du système sera comparée à elle pour dire si elle est correcte ou non.  Vous regrouperez toutes ces entrées dans une liste totale de 2368 éléments appelée `items_with_features_A`.

En partant de la liste des phrases appelée `sentences`(préparée plus haut), veuillez générer ici cette liste, en vous aidant si nécessaire du livre NLTK.

In [ ]:
# Veuillez répondre ici à la question.

print(len(items_with_features_A))
print(items_with_features_A[151:154])

On souhaite maintenant entraîner un classifieur sur une partie des données, et le tester sur une autre.  Typiquement, on peut garder 80% des données pour l'entraînement et utiliser les 20% restants pour l'évaluation.  Naturellement, on fait cette division séparément pour chaque sens, pour que les deux ensembles contiennent les mêmes proportions de sens que l'ensemble de départ.  (On parle de "*stratified split*").  Nous ferons cette division aléatoirement, une seule fois, mais typiquement on la fait plusieurs fois et on fait la moyenne des scores obtenus ("*cross-validation*").  Il y a des packages qui font cela automatiquement (p.ex. `scikit-learn`).

Veuillez maintenant deux sous-ensembles de `items_with_features_A` selon ces indications, appelés `iwf_A_train` et `iwf_A_test`.


In [ ]:
from random import shuffle

In [ ]:
iwf_A_train = []
iwf_A_test  = []
# Veuillez répondre ici à la question.

print(len(iwf_A_train), ' ', len(iwf_A_test))
print(iwf_A_test[:2], iwf_A_test[-2:])

Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_A_train` et la tester sur `iwf_A_train` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score global que vous obtenez, et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs, et pouvez-vous expliquer cet affichage ?

*Note : vous pouvez choisir de générer plusieurs fois aléatoirement les deux sous-ensembles, et voir comment les scores varient.*

*Note 2 : il serait possible de diviser les données en 3, et choisir la meilleur taille de la fenêtre sur un ensemble de développement, différent de celui de test final.*

In [ ]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.

classifier = nltk.NaiveBayesClassifier.train(train_set) [1]
print(nltk.classify.accuracy(classifier, devtest_set))

On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (cherchez dans le [livre NLTK](https://www.nltk.org/book/ch06.html) comment), et comptabiliser les prédictions correctes pour chaque sens.  Suggestion : inspirez-vous de `evaluate_wsd`, mais appliquez-là seulement aux données `iwf_A_test`.  Veuillez écrire une fonction nommée ainsi : `evaluate_wsd_supervised(classifier, items_with_features)`.

In [ ]:
# Veuillez répondre ici à la question.


### 3.B. Présence de mots indicateurs

Une deuxième façon d'encoder les traits lexicaux est de constituer un vocabulaire avec les mots qui apparaissent dans le voisinage de *interest* et de définir ces mots comme traits.  Par conséquent, pour chaque occurrence de *interest*, vous allez extraire la valeur de ces traits sous la forme :
```
[{('rate' : True), ('in' : False), ...}, 1]
```
où *'rate'*, *'in'* sont les mots du vocabulaire, True/False indiquent leur présence/absence autour de l'occurrence de *interest* qui est décrite, et le dernier nombre est le sens, entre 1 et 6.

Pour commencer, en partant de `sentences` et en fixant la taille de la fenêtre, veuillez constituer la liste de tous les mots observés autour de tous les voisinages de toutes les occurrences de *interest*.

In [ ]:
word_list = []
# Veuillez répondre ici à la question.

print(len(word_list))
print(word_list[:50])

En utilisant un objet `nltk.FreqDist`, veuillez sélectioner les 500 mots les plus fréquents (vous pourrez aussi faire varier ce nombre), dans une liste appelée `vocabulary`.  À votre avis, est-ce une bonne idée d'enlever les *stopwords* de cette liste pour construire les traits ?

In [ ]:
# Veuillez répondre ici à la question.

print(vocabulary[:50])

Veuillez maintenant créer l'ensemble total de données formatées, en convertissant chaque phrase contenant une occurrence de *interest* à un dictionnaire de traits/valeurs (suivi du numéro du sens), comme exemplifié au début de cette section 3B.  Cet ensemble sera appelé `items_with_features_B`.

In [ ]:
items_with_features_B = []
# Veuillez répondre ici à la question.

print(len(items_with_features_B))

Comme dans la section 3A, veuillez créer maintenant deux sous-ensembles de `items_with_features_B` appelés `iwf_B_train` (80% des items) et `iwf_B_test` (20% des items), avec une sélection aléatoire mais stratifiée.

In [ ]:
iwf_B_train = []
iwf_B_test  = []
# Veuillez répondre ici à la question.

print(len(iwf_B_train), ' ', len(iwf_B_test))

Comme pour la section 3A, veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_B_train` et la tester sur `iwf_B_train`.  Veuillez tester ce classifieur globalement, puis pour chacun des six sens (classes) avec la méthode `evaluate_wsd_supervised` du 3A.

En expérimentant avec différentes largeurs de fenêtres et tailles du vocabulaire, quel est le meilleur score que vous obtenez, et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs ?

In [ ]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.


In [ ]:
# Veuillez recopier ici en conclusion les scores par sens des quatre 
# expériences, pour pouvoir les comparer d'un coup d'oeil.


## Fin du laboratoire

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, et sauvegarder le résultat.  
Comprimez la feuille dans un fichier `.zip` et soumettez-le sur Cyberlearn.